# Combining Datasets - Intro

In this chapter, you will learn how to combine data from different sources. This mostly involves a so called **join** (or, sometimes, "merge") of tables with different columns. In that case, rows from one table are *matched* using columns from both tables that contain common "identifiers" (e.g. a compy ID or name, a period indicator such as *year* etc.).

In addition, we will cover the case of a **union** (or, sometimes, "append" or "concat(enation)" operation) where the rows of one table are *added* to the rows of another table. These tables typically have the *same* columns (or at least they share a common subset of columns).

# Preparations

In [1]:
import pandas as pd

pd.set_option("display.max_columns", 500)

# Joins (`pd.DataFrame.merge()`)

## Overview

When two tables are combined by matching their rows this is most commonly called a *join*, especially in the context of relational databases and SQL. In some programming languages, including *Python*, *R*, and *Stata* this operation is also called a *merge*.

For example, we might want to merge a dataset containing time-invariant data on countries (e.g. region or currency) with time-variant data like population or GDP.

When two tables are *joined*, it is common to call one table the *'left'* table and the other the *'right'* table. Accordingly, there are **four types of joins** :
1. ) **inner join**: keep rows that have matching values in both tables.

2. ) **left join**: keep all rows from the left table, and the matched rows from the right table.

3. ) **right join**: keep all rows from the right table, and the matched rows from the left table.

4. ) **full join**: keep all rows from both tables.

When matching rows from one table to another, there can be **three different relations** between the rows from both tables:

1. ) **1:1**

    * One row in Table A matches one row in Table B OR there is no match.
    * e.g. two tables containing different time-invariant data on countries

2. ) **1:m**

	* One row in Table A matches one or more row in Table B OR none at all.
    * e.g. the aforementioned example of Table A containing time-invariant and Table B containing data for several years per country.

3. ) **m:m**

    * One or more rows in Table A match one or more rows in Table B OR there is no match at all.
    * e.g. Firms can belong to several industries and industries typically contain several firms.
    * Note: direct m:m merges are somewhat rare in practice. Typically, one has to perform two or more 1:m or n:1 merges in sequence, instead.
    * The special case where *each* row of one table is matched with *each* row of the other table. This is called a **cross-join**.

See the file **"join_examples.xlsx"** in the raw-data folder for illustrated examples.

## The `pd.DataFrame.merge()` method

The `pd.DataFrame.merge()` method can be used to perform database-style joins.

The join is done on columns or indexes. If joining columns on columns, the DataFrame indexes will be ignored. Otherwise if joining indexes on indexes or indexes on a column or columns, the index will be passed on.

In the following, we will see the typical join situations in action.

Also see [https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html) and [https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#merging-join](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#merging-join)

## 1:1 joins

### Loading the two tables

In [2]:
df_left = pd.read_excel("../../data/raw/join_examples.xlsx", sheet_name="JOIN_11_Table_A")
df_left.head(10)

,countryname,countrycode,source_table
0,Germany,DEU,A
1,France,FRA,A
2,United Kingdom,GBR,A
3,Iraq,IRQ,A
4,United States,USA,A


In [3]:
df_right = pd.read_excel("../../data/raw/join_examples.xlsx", sheet_name="JOIN_11_Table_B")
df_right.head(10)

,countrycode,currencyunit,incomegroup,source_table
0,ASM,U.S. dollar,Upper middle income,B
1,BWA,Botswana pula,NaN,B
2,DEU,Euro,High income,B
3,FRA,Euro,High income,B
4,GBR,Pound sterling,High income,B


### Inner join

`how` and `on` define the kind of join and the common identifiers.

In [4]:
df_join = df_left.merge(df_right, how="inner", on="countrycode")
df_join

,countryname,countrycode,source_table_x,currencyunit,incomegroup,source_table_y
0,Germany,DEU,A,Euro,High income,B
1,France,FRA,A,Euro,High income,B
2,United Kingdom,GBR,A,Pound sterling,High income,B


It  is good practice to validate the integrity of the assumed relation between the tables' rows, here '1:1'. `.merge()` can do this by passing the appropriate `validate` argument (note, that the 1:1 relation is a special case of the 1:m relation!):

In [5]:
# explicitly expect a 1:1 relation
df_join = df_left.merge(df_right, how="inner", on="countrycode", validate="1:1")

# expect a 1:m relation instead:
df_join = df_left.merge(df_right, how="inner", on="countrycode", validate="1:m")

In [6]:
# Let's force an error by duplicating the left table
df_left_times_two = pd.concat([df_left, df_left])
try:
    df_left_times_two.merge(df_right, how="inner", on="countrycode", validate="1:1")
except Exception as e:  # noqa: BLE001
    print(f"Error: {e}")

Error: Merge keys are not unique in left dataset; not a one-to-one merge
Duplicates in left:
 countrycode
        DEU
        FRA
        GBR
        IRQ
        USA ...


You should also check which rows from the two tables are **not matched**:

In [7]:
df_left.loc[~df_left["countrycode"].isin(df_join["countrycode"])]

,countryname,countrycode,source_table
3,Iraq,IRQ,A
4,United States,USA,A


In [8]:
df_right.loc[~df_right["countrycode"].isin(df_join["countrycode"])]

,countrycode,currencyunit,incomegroup,source_table
0,ASM,U.S. dollar,Upper middle income,B
1,BWA,Botswana pula,NaN,B


### Left join

In [9]:
df_join = df_left.merge(df_right, how="left", on="countrycode", validate="1:1")
df_join

,countryname,countrycode,source_table_x,currencyunit,incomegroup,source_table_y
0,Germany,DEU,A,Euro,High income,B
1,France,FRA,A,Euro,High income,B
2,United Kingdom,GBR,A,Pound sterling,High income,B
3,Iraq,IRQ,A,NaN,NaN,NaN
4,United States,USA,A,NaN,NaN,NaN


In [10]:
# still missing a few rows from the right
df_right.loc[~df_right["countrycode"].isin(df_join["countrycode"])]

,countrycode,currencyunit,incomegroup,source_table
0,ASM,U.S. dollar,Upper middle income,B
1,BWA,Botswana pula,NaN,B


### Right join

In [11]:
df_join = df_left.merge(
    df_right, how="right", on="countrycode", validate="1:1", indicator="check_merge"
)
df_join

,countryname,countrycode,source_table_x,currencyunit,incomegroup,source_table_y,check_merge
0,NaN,ASM,NaN,U.S. dollar,Upper middle income,B,right_only
1,NaN,BWA,NaN,Botswana pula,NaN,B,right_only
2,Germany,DEU,A,Euro,High income,B,both
3,France,FRA,A,Euro,High income,B,both
4,United Kingdom,GBR,A,Pound sterling,High income,B,both


In [12]:
df_left.loc[~df_left["countrycode"].isin(df_join["countrycode"])]

,countryname,countrycode,source_table
3,Iraq,IRQ,A
4,United States,USA,A


### Outer join

The outer join is very useful because it is easier to see which rows have not been matched - a fact that you should **always be aware** of! Therefore, in practice, it is recommended to start with the outer join and then drop rows in a *controlled* fashion according to the desired final result. For example, here we start with an outer join but reduce the final result to an inner join, explicitly taking account of the dropped rows on the way:

In [13]:
df_join = df_left.merge(df_right, how="outer", on="countrycode", validate="1:1")
df_join

,countryname,countrycode,source_table_x,currencyunit,incomegroup,source_table_y
0,NaN,ASM,NaN,U.S. dollar,Upper middle income,B
1,NaN,BWA,NaN,Botswana pula,NaN,B
2,Germany,DEU,A,Euro,High income,B
3,France,FRA,A,Euro,High income,B
4,United Kingdom,GBR,A,Pound sterling,High income,B
5,Iraq,IRQ,A,NaN,NaN,NaN
6,United States,USA,A,NaN,NaN,NaN


In [14]:
# Validate that we have not lost any rows
# this time, we make use of the `indicator` option:
df_join = df_left.merge(
    df_right, how="outer", on="countrycode", validate="1:1", indicator="check_merge"
)
df_join["check_merge"].value_counts()

check_merge
both          3
left_only     2
right_only    2
Name: count, dtype: int64

In [15]:
# Rows not matched from the left table
df_join[df_join["check_merge"] == "left_only"]

,countryname,countrycode,source_table_x,currencyunit,incomegroup,source_table_y,check_merge
5,Iraq,IRQ,A,NaN,NaN,NaN,left_only
6,United States,USA,A,NaN,NaN,NaN,left_only


In [16]:
# Rows not matched from the right table
df_join[df_join["check_merge"] == "right_only"]

,countryname,countrycode,source_table_x,currencyunit,incomegroup,source_table_y,check_merge
0,NaN,ASM,NaN,U.S. dollar,Upper middle income,B,right_only
1,NaN,BWA,NaN,Botswana pula,NaN,B,right_only


In [17]:
# Rows equivalent to inner join
df_join[df_join["check_merge"] == "both"]

,countryname,countrycode,source_table_x,currencyunit,incomegroup,source_table_y,check_merge
2,Germany,DEU,A,Euro,High income,B,both
3,France,FRA,A,Euro,High income,B,both
4,United Kingdom,GBR,A,Pound sterling,High income,B,both


## Exercise 1

1. Load the first sheet of the Excel file "wdi_reduced.xlsx" into a pandas DataFrame (see [here](https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html) for help with `pandas.read_excel()`)

2. Load the annual CO2 emmissions per capita from the file "wdi_EN_ATM_CO2E_PC.csv".

3. (Inner) join the CO2 emmissions with our main dataset from "wdi_reduced". Make sure to validate the expected 1:1 relation and to check for any unmatched rows from both tables.

4. BONUS: Load the annual unemployment rates from the file "wdi_SL_UEM_TOTL_ZS.xlsx".

5. BONUS: Reshape the annual unemployment rates dataset to long format containg one row per *country* and *year*.

6. BONUS: (Inner) join it with our main dataset from "wdi_reduced". Make sure to validate the expected 1:1 relation and to check for any unmatched rows from both tables.

## 1:m joins

### Loading the two tables

In [18]:
df_left = pd.read_excel("../../data/raw/join_examples.xlsx", sheet_name="JOIN_1m_Table_A")
df_left.head(10)

,countryname,countrycode,source_table
0,Germany,DEU,A
1,France,FRA,A
2,United Kingdom,GBR,A
3,Iraq,IRQ,A
4,United States,USA,A


In [19]:
df_right = pd.read_excel("../../data/raw/join_examples.xlsx", sheet_name="JOIN_1m_Table_B")
df_right.head(10)

,countrycode,year,NY_GDP_MKTP_CD,source_table
0,ASM,2002,0.514000,B
1,ASM,2001,0.000000,B
2,ASM,2003,0.527000,B
3,BWA,2003,7.511582,B
4,BWA,2001,5.489608,B
5,BWA,2002,5.438857,B
6,DEU,2002,2079.136081,B
7,DEU,2003,2505.733634,B
8,DEU,2001,1950.648770,B
9,FRA,2001,1382.218345,B


### Inner join

In [20]:
# expecting a 1:1 relation will fail
# df_join = df_left.merge(df_right, how="inner", on="countrycode", validate='1:1')

# explicitly expect a 1:m relation
df_join = df_left.merge(df_right, how="inner", on="countrycode", validate="1:m")
df_join

,countryname,countrycode,source_table_x,year,NY_GDP_MKTP_CD,source_table_y
0,Germany,DEU,A,2002,2079.136081,B
1,Germany,DEU,A,2003,2505.733634,B
2,Germany,DEU,A,2001,1950.648770,B
3,France,FRA,A,2001,1382.218345,B
4,France,FRA,A,2002,1500.337851,B
5,France,FRA,A,2003,1848.124153,B
6,United Kingdom,GBR,A,2003,2028.488163,B
7,United Kingdom,GBR,A,2001,1613.034403,B
8,United Kingdom,GBR,A,2002,1757.571942,B


### Left join

In [21]:
df_join = df_left.merge(df_right, how="left", on="countrycode", validate="1:m")
df_join

,countryname,countrycode,source_table_x,year,NY_GDP_MKTP_CD,source_table_y
0,Germany,DEU,A,2002.0,2079.136081,B
1,Germany,DEU,A,2003.0,2505.733634,B
2,Germany,DEU,A,2001.0,1950.648770,B
3,France,FRA,A,2001.0,1382.218345,B
4,France,FRA,A,2002.0,1500.337851,B
5,France,FRA,A,2003.0,1848.124153,B
6,United Kingdom,GBR,A,2003.0,2028.488163,B
7,United Kingdom,GBR,A,2001.0,1613.034403,B
8,United Kingdom,GBR,A,2002.0,1757.571942,B
9,Iraq,IRQ,A,NaN,NaN,NaN


### Right join

In [22]:
df_join = df_left.merge(df_right, how="right", on="countrycode", validate="1:m")
df_join

,countryname,countrycode,source_table_x,year,NY_GDP_MKTP_CD,source_table_y
0,NaN,ASM,NaN,2002,0.514000,B
1,NaN,ASM,NaN,2001,0.000000,B
2,NaN,ASM,NaN,2003,0.527000,B
3,NaN,BWA,NaN,2003,7.511582,B
4,NaN,BWA,NaN,2001,5.489608,B
5,NaN,BWA,NaN,2002,5.438857,B
6,Germany,DEU,A,2002,2079.136081,B
7,Germany,DEU,A,2003,2505.733634,B
8,Germany,DEU,A,2001,1950.648770,B
9,France,FRA,A,2001,1382.218345,B


### Outer join

In [23]:
df_join = df_left.merge(
    df_right, how="outer", on="countrycode", validate="1:m", indicator="check_merge"
)
df_join

,countryname,countrycode,source_table_x,year,NY_GDP_MKTP_CD,source_table_y,check_merge
0,NaN,ASM,NaN,2002.0,0.514000,B,right_only
1,NaN,ASM,NaN,2001.0,0.000000,B,right_only
2,NaN,ASM,NaN,2003.0,0.527000,B,right_only
3,NaN,BWA,NaN,2003.0,7.511582,B,right_only
4,NaN,BWA,NaN,2001.0,5.489608,B,right_only
5,NaN,BWA,NaN,2002.0,5.438857,B,right_only
6,Germany,DEU,A,2002.0,2079.136081,B,both
7,Germany,DEU,A,2003.0,2505.733634,B,both
8,Germany,DEU,A,2001.0,1950.648770,B,both
9,France,FRA,A,2001.0,1382.218345,B,both


In [24]:
# Check unmatched rows (counts)
df_join["check_merge"].value_counts()

check_merge
both          9
right_only    6
left_only     2
Name: count, dtype: int64

## m:m join

### Loading the two tables

In [25]:
df_left = pd.read_excel("../../data/raw/join_examples.xlsx", sheet_name="JOIN_mm_Table_A")
df_left.head(10)

,countryname,countrycode,region,source_table
0,Germany,DEU,Europe & Central Asia,A
1,France,FRA,Europe & Central Asia,A
2,United Kingdom,GBR,Europe & Central Asia,A
3,Iraq,IRQ,Middle East & North Africa,A
4,United States,USA,North America,A
5,Canada,CAN,North America,A
6,Brasil,BRA,Latin America & Caribbean,A


In [26]:
df_right = pd.read_excel("../../data/raw/join_examples.xlsx", sheet_name="JOIN_mm_Table_B")
df_right.head(10)

,region,datasource,regionalindex,source_table
0,Europe & Central Asia,Best datasource,0.565655,B
1,Europe & Central Asia,Medium quality datasource,0.671547,B
2,Middle East & North Africa,Best datasource,NaN,B
3,Middle East & North Africa,Medium quality datasource,NaN,B
4,North America,Best datasource,NaN,B
5,North America,Medium quality datasource,0.810120,B
6,South Asia,Best datasource,0.539802,B
7,South Asia,Medium quality datasource,0.832581,B
8,Sub-Saharan Africa,Best datasource,NaN,B
9,Sub-Saharan Africa,Medium quality datasource,0.627779,B


### Outer join

In [27]:
# assuming 1:1 or 1:m would fail
# df_left.merge(df_right, how="outer", on="region", validate='1:1')
# df_left.merge(df_right, how="outer", on="region", validate='1:m')

# instead, validate a m:m relation:
df_join = df_left.merge(df_right, how="outer", on="region", validate="m:m", indicator="check_merge")
df_join

,countryname,countrycode,region,source_table_x,datasource,regionalindex,source_table_y,check_merge
0,Germany,DEU,Europe & Central Asia,A,Best datasource,0.565655,B,both
1,Germany,DEU,Europe & Central Asia,A,Medium quality datasource,0.671547,B,both
2,France,FRA,Europe & Central Asia,A,Best datasource,0.565655,B,both
3,France,FRA,Europe & Central Asia,A,Medium quality datasource,0.671547,B,both
4,United Kingdom,GBR,Europe & Central Asia,A,Best datasource,0.565655,B,both
5,United Kingdom,GBR,Europe & Central Asia,A,Medium quality datasource,0.671547,B,both
6,Brasil,BRA,Latin America & Caribbean,A,NaN,NaN,NaN,left_only
7,Iraq,IRQ,Middle East & North Africa,A,Best datasource,NaN,B,both
8,Iraq,IRQ,Middle East & North Africa,A,Medium quality datasource,NaN,B,both
9,United States,USA,North America,A,Best datasource,NaN,B,both


### Drop unmatched rows in a controlled fashion (ending up a 'left' join)

In [28]:
# Note rows not matched and drop them
print(df_join["check_merge"].value_counts())

df_join = df_join[df_join["check_merge"] == "both"]
# the dropna may have created a view on the original DataFrame
print(f"Rows after producing left join ('manually'): {len(df_join)}")
df_join

check_merge
both          12
right_only     4
left_only      1
Name: count, dtype: int64
Rows after producing left join ('manually'): 12


,countryname,countrycode,region,source_table_x,datasource,regionalindex,source_table_y,check_merge
0,Germany,DEU,Europe & Central Asia,A,Best datasource,0.565655,B,both
1,Germany,DEU,Europe & Central Asia,A,Medium quality datasource,0.671547,B,both
2,France,FRA,Europe & Central Asia,A,Best datasource,0.565655,B,both
3,France,FRA,Europe & Central Asia,A,Medium quality datasource,0.671547,B,both
4,United Kingdom,GBR,Europe & Central Asia,A,Best datasource,0.565655,B,both
5,United Kingdom,GBR,Europe & Central Asia,A,Medium quality datasource,0.671547,B,both
7,Iraq,IRQ,Middle East & North Africa,A,Best datasource,NaN,B,both
8,Iraq,IRQ,Middle East & North Africa,A,Medium quality datasource,NaN,B,both
9,United States,USA,North America,A,Best datasource,NaN,B,both
10,United States,USA,North America,A,Medium quality datasource,0.810120,B,both


### Drop NA values in the column of interest

If a data sosurce does not provide a `regionalindex` for a region, we can ignore it

In [29]:
df_join = df_join.dropna(subset=["regionalindex"])
# the following line is no longer necessary in pandas 3.0+ because of Copy-on-Write:
# df_join = df_join.copy()

# or:
# df_join = df_join[df_join["regionalindex"].notna()]

df_join

,countryname,countrycode,region,source_table_x,datasource,regionalindex,source_table_y,check_merge
0,Germany,DEU,Europe & Central Asia,A,Best datasource,0.565655,B,both
1,Germany,DEU,Europe & Central Asia,A,Medium quality datasource,0.671547,B,both
2,France,FRA,Europe & Central Asia,A,Best datasource,0.565655,B,both
3,France,FRA,Europe & Central Asia,A,Medium quality datasource,0.671547,B,both
4,United Kingdom,GBR,Europe & Central Asia,A,Best datasource,0.565655,B,both
5,United Kingdom,GBR,Europe & Central Asia,A,Medium quality datasource,0.671547,B,both
10,United States,USA,North America,A,Medium quality datasource,0.810120,B,both
12,Canada,CAN,North America,A,Medium quality datasource,0.810120,B,both


### Dropping duplicates (in terms of countries)

As there are two possible `datasource`s for our `regionalindex`, we have created duplicates entries for the countries. We would like to keep only one row per country, preferably with the 'Useful Ratings Group' as `datasource`, but, if it is not available *and* 'Another Ratings Agency' is, then we use the latter.

The method `drop_duplicates()` can be used to drop rows that have duplicate entries according to defined `subset` of columns. By default, the 'first' row of a given duplicate set is retained (`keep`='first').

In other words, the order in the dataset decides which rows are to be kept! Here, we want to sort the dataset, so that (if available), the best datasource is 'on top'.

In [30]:
# ensure the desired order using a helper column

df_join.loc[:, "priority"] = (
    100  # start by initializing a sorting column with something that will be sorted to the end of the table
)
# now ensure that the best datasource is used when available:
df_join.loc[
    (df_join.datasource == "Useful Ratings Group"),
    "priority",
] = 0
# the second best option is the other
df_join.loc[
    (df_join.datasource == "Another Ratings Agency"),
    "priority",
] = 1
# now sort accordingly
df_join = df_join.sort_values("priority")
df_join

,countryname,countrycode,region,source_table_x,datasource,regionalindex,source_table_y,check_merge,priority
0,Germany,DEU,Europe & Central Asia,A,Best datasource,0.565655,B,both,100
1,Germany,DEU,Europe & Central Asia,A,Medium quality datasource,0.671547,B,both,100
2,France,FRA,Europe & Central Asia,A,Best datasource,0.565655,B,both,100
3,France,FRA,Europe & Central Asia,A,Medium quality datasource,0.671547,B,both,100
4,United Kingdom,GBR,Europe & Central Asia,A,Best datasource,0.565655,B,both,100
5,United Kingdom,GBR,Europe & Central Asia,A,Medium quality datasource,0.671547,B,both,100
10,United States,USA,North America,A,Medium quality datasource,0.810120,B,both,100
12,Canada,CAN,North America,A,Medium quality datasource,0.810120,B,both,100


In [31]:
# finally, drop the duplicates
df_join = df_join.drop_duplicates(subset="countrycode")
df_join

,countryname,countrycode,region,source_table_x,datasource,regionalindex,source_table_y,check_merge,priority
0,Germany,DEU,Europe & Central Asia,A,Best datasource,0.565655,B,both,100
2,France,FRA,Europe & Central Asia,A,Best datasource,0.565655,B,both,100
4,United Kingdom,GBR,Europe & Central Asia,A,Best datasource,0.565655,B,both,100
10,United States,USA,North America,A,Medium quality datasource,0.810120,B,both,100
12,Canada,CAN,North America,A,Medium quality datasource,0.810120,B,both,100


# Unions - the `pd.concat()` function

## `pd.concat()`

The `pd.concat()` function is a very flexible tool. It can be used for combining datasets both in a `.merge()`-like fashion and for creating "unions", i.e. row-wise appending operations. The most useful feature is its capability of handling **lists of `DataFrame`s** instead of only two at a time!

Also see [https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html#pandas.concat](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html#pandas.concat) and [https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#concatenating-objects](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#concatenating-objects)


In [32]:
# create a DataFrame
df_one = pd.DataFrame({"key": ["A", "B"], "value": [10, 20]})
df_one

,key,value
0,A,10
1,B,20


In [33]:
# create another DataFrame
df_two = pd.DataFrame({"key": ["C", "D"], "value": [30, 40]})
df_two

,key,value
0,C,30
1,D,40


In [34]:
# let us create yet another DataFrame
df_three = pd.DataFrame({"key": ["E", "F", "G"], "value": [50, 60, 70]})
df_three

,key,value
0,E,50
1,F,60
2,G,70


In [35]:
# now concatenate
pd.concat([df_one, df_two, df_three])

,key,value
0,A,10
1,B,20
0,C,30
1,D,40
0,E,50
1,F,60
2,G,70


In [36]:
# create a new index
pd.concat([df_one, df_two, df_three], ignore_index=True)

,key,value
0,A,10
1,B,20
2,C,30
3,D,40
4,E,50
5,F,60
6,G,70


## Exercise 2

1. Load the first sheet of the Excel file "wdi_reduced.xlsx" into a pandas DataFrame (see [here](https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html) for help with `pandas.read_excel()`)
2. Split the dataset wdi_prepared into two DataFrames (one for years < 2000 and one for years >= 2000).
3. Then concatenate the two dataframes into a new combined dataframe. Check that the original dataframe and the newly created dataframe have the same length.
4. BONUS: split the dataset into one DataFrame for each year and store the DataFrames in a list.
5. BONUS: combine the list datasets using the `pd.concat()` function.